In [1]:
import pandas as pd
import glob
import os
import string
import numpy as np
import spacy
# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# nltk
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# 1 Import Data

## 1.1 Import Left-Wing Reddit Data

In [2]:
# get CSV file list from a foder
path = './radLeftSD/'
csv_files = glob.glob(path + "*.csv")

#read each CSV file into dataframe
df_list = (pd.read_csv(file) for file in csv_files)

leftwing_data = pd.concat(df_list, ignore_index=True)
leftwing_data

Unnamed: 0                                              Title  \
0              0  if you have a moment can you please fill out t...   
1              1  WE DECLARE 17 DAYS OF INTERNATIONAL CELEBRATIO...   
2              2  YJA Star: We will win the battle and make the ...   
3              3  Protesters in the Magway Region of Myanmar gua...   
4              4            Alkan: Guerrilla is always volunteering   
...          ...                                                ...   
5315         957  Donald Trump announcing that Gender would be d...   
5316         958                         was told to post this here   
5317         959                        Being trans is NOT a fetish   
5318         960  The State will not protect you. Buy a gun. Lea...   
5319         961                              Trans Yugoslavia flag   

                                              Post Text       ID  Score  \
0                                                   NaN   sy1c7u     31   
1     \nTo counter HM King Charles III's declaration...   xawd3v     85   
2                                                   NaN  11jw75t     41   
3                                                   NaN  11jxcdf      4   
4                                                   NaN  11jztjh      1   
...                                                 ...      ...    ...   
5315                                                NaN   9q8qta    178   
5316                                                NaN   9q7xy4    185   
5317  Fetishism embodies all of the worst aspects of...   9q9f88     41   
5318                                                NaN   9q4wc4    187   
5319                                                NaN   9q8k4n     37   

      Total Comments                                           Post URL  \
0                 21  https://firebasehostingproxy.page.link/1988868...   
1                  7  https://www.reddit.com/r/LateStageColonialism/...   
2                  1                https://i.redd.it/klrxl3urs3ma1.jpg   
3                  0                https://i.redd.it/kvlggp7m24ma1.jpg   
4                  0  https://www.nuceciwan119.xyz/en/2023/03/alkan-...   
...              ...                                                ...   
5315               3  https://images.dailykos.com/images/276245/stor...   
5316               2                https://i.redd.it/9ilwo6qmamt11.jpg   
5317               2  https://www.reddit.com/r/LateStageGenderBinary...   
5318              17  https://twitter.com/LA_SocialistRA/status/1054...   
5319               9  https://cdn.discordapp.com/attachments/4943158...   

                                          Post Comments  
0     ['this is the last full day the survey will be...  
1               ['Does this include the Danish queen?']  
2     ['"We as YJA Star forces fighting for the nobl...  
3                                                    []  
4                                                    []  
...                                                 ...  
5315  ['Hope we can stop this asshole before he make...  
5316  ["It would be hilarious if it wasn't true.", '...  
5317  ['Wow. I like...\n\n&#x200B;\n\nHmmmm. Half ag...  
5318  ['Also go beyond weapons (though I 100% agree ...  
5319  ['Market socialism though 😬', "I made one some...  

[5320 rows x 8 columns]

## 1.2 Import Right-Wing Patriots.Win Data

In [3]:
# get CSV file list from a foder
path = './altRightSD/'
csv_files = glob.glob(path + "*.csv")

#read each CSV file into dataframe
df_list = (pd.read_csv(file) for file in csv_files)

rightwing_data = pd.concat(df_list, ignore_index=True)
rightwing_data

Unnamed: 0                                              Title  \
0             0   \nTRUMP TWEET. DADDY SAYS BE IN DC ON JAN. 6TH\n   
1             1                             \nThis is how we won\n   
2             2  \nRaise your hand if you check this site as so...   
3             3         \nUpvote if Fox News is dead to you too.\n   
4             4  \nNUMBER OF PEDES THAT DID NOT WATCH THE SHAM ...   
..          ...                                                ...   
719         236                                          \nTRUTH\n   
720         237  \nSuspect in Shooting of Jews in L.A. Was Out ...   
721         238  \n'I’m Begging You Not to Drink the Water’ in ...   
722         239  \n23.1 percent of Californians over the age of...   
723         240           \nCockpit photo of Chinese Spy Balloon\n   

                                              Post URL  \
0    /p/11R4q2aptJ/trump-tweet-daddy-says-be-in-dc-/c/   
1                      /p/GIEYMyjg/this-is-how-we-won/   
2      /p/11Q8XPAOyg/raise-your-hand-if-you-check-thi/   
3      /p/11Q8O0hnaE/upvote-if-fox-news-is-dead-to-yo/   
4      /p/11S13uq5gt/number-of-pedes-that-did-not-wat/   
..                                                 ...   
719                             /p/16aT36KnaQ/truth/c/   
720  /p/16aANBPLca/suspect-in-shooting-of-jews-in-l/c/   
721  /p/16aAN99eZo/im-begging-you-not-to-drink-the-/c/   
722  /p/16aAN7zKa9/231-percent-of-californians-over/c/   
723  /p/16aT3AsvAM/cockpit-photo-of-chinese-spy-bal/c/   

                                         Post Comments  
0    ["\nWell, shit. We've got marching orders, fre...  
1    ["\nThey wouldn't have to censor us if we were...  
2    ["\nI've woken up at night to check TDW!\n", '...  
3    ['\nAlex Jones just broke news Tucker is heavi...  
4    ["\nIt's on the front page of Twitch and chat ...  
..                                                 ...  
719  ['\nSending this one to my 22 year old daughte...  
720  ['\nWho cares if they were Jews? If we can poi...  
721  ['\nTruly is sickening the incompetence at our...  
722  ["\nThey can't read that sentence... because i...  
723  ["\nIt is pronounced 'BAROOON'\n", '\nlow-arti...  

[724 rows x 4 columns]

## 1.3 A brief note on how I fucked and am trying to unfuck the data
Essentially, the "Post Comments" column for each data frame was expected to consist of a clean list of posts for each entry. However, these lists only contain ONE BIG STRING consisting of all of the posts, which can be seen below: if you try to index into the first entry (e.g., the first list) you only get characters returned

In [4]:
rightwing_data["Post Comments"][0] # here is the first entry # 

'["\\nWell, shit. We\'ve got marching orders, frens.\\n", "\\nminecraft doesn\'t allow pickaxes on flying machines\\n", "\\nminecraft doesn\'t allow pickaxes on flying machines\\n", "\\nminecraft doesn\'t allow pickaxes on flying machines\\n", "\\nI\'m told DC is already full of hoes though.\\n", \'\\n[Dan Crenshaw has entered the chat]\\n\', \'\\n"I swear i never entered the chat my account was hacked"\\n\', \'\\nPatriots join and get in DONALD ARMY, state and nation chat – Organize, Go Together. We are organizing for Patriots to go to DC 1/6/21. We need local leaders to help recruit.\\nExamples of things being coordinated:\\nSharing of room/board and locations of safe places to book Traveling together Meetup Spots General Safety Events and locations Communication Ladies we can and will connect you with other women so you are safe.\\nInstructions (Do this from Desktop as it is too much for mobile):\\nNOTE: You should set up a private protonmail account with no personal information for

In [5]:
rightwing_data["Post Comments"][0][0] # indexing into this list, you get a char >:( # 

'['

In [6]:
leftwing_data["Post Comments"][0][0] # you can see this is also the case for leftwing data #

'['

## 1.4 Unfucking the data: preprocessing to try to regain the integrity of individual posts
As you can see, different posts are delimited by '\\n', so my hope is to try to split these large strings into separate posts this way

In [7]:
len(rightwing_data["Post Comments"][0].split('\\n'))

1897

Base preprocessing technique that can be applied to any given string

In [8]:
def base_preprocess(input_str):
    rez = []
    punctuation = string.punctuation + '…' + '—' + '\n'
    filter_punct = str.maketrans('','',punctuation)
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    tokens = input_str.split()
    for token in tokens:
        if token.isnumeric(): 
            continue
        token = token.lower().translate(filter_punct)
        if token not in stop_words:
            token = lemmatizer.lemmatize(token)
            rez.append(token)
    return rez

method that iterates through each df row and produces an actual list of comments for each df, that list being preprocessed

In [9]:
def process_df(df):
    dict_obj = {}
    for index, row in df.iterrows():
        key = ' '.join(base_preprocess(row["Title"]))
        dict_obj[key] = []
        strs = row["Post Comments"].split("\\n")
        for entry in strs:
            dict_obj[key].append(base_preprocess(entry))
    return dict_obj

## 1.5 Dictionaries for each title-comment section setup

In [10]:
rightwing_dict = process_df(rightwing_data)

In [11]:
leftwing_dict = process_df(leftwing_data)

# 2 Topic Modeling: LDA

In [12]:
# vis imports
import pyLDAvis
import pyLDAvis.gensim

## 2.1 LDA Topic Model for rightwing data 

In [13]:
# turn each posts dict item into a single, word vector
def gen_words(key, posts_dict):
    final = [word for word in key.split()]
    for entry in posts_dict[key]:
        for word in entry:
            if (word == ''):
                continue
            final.append(word)
    return (final)

In [14]:
def data_words(dictionary):
    foo = []
    for key in dictionary.keys():
        foo.append(gen_words(key, dictionary))
    return foo

rightwing_dw = data_words(rightwing_dict)
# leftwing_dw = data_words(leftwing_dict)

In [15]:
def freq_vec(data_words):
    id2word = corpora.Dictionary(data_words)
    corpus = []
    for text in data_words:
        new = id2word.doc2bow(text)
        corpus.append(new)
    return id2word, corpus

rightwing_id2word, rightwing_corpus = freq_vec(rightwing_dw)

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=rightwing_corpus,
                                           id2word=rightwing_id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, rightwing_corpus, rightwing_id2word, mds="mmds", R=30)
vis

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
27     0.190495  0.445881       1        1  36.509895
0      0.364878  0.351107       2        1  35.928843
2      0.024215  0.494517       3        1  27.323098
11    -0.151842  0.042904       4        1   0.162978
7      0.026841 -0.081808       5        1   0.048067
16    -0.015315 -0.052604       6        1   0.007278
9     -0.014686 -0.046580       7        1   0.006982
5     -0.022345 -0.053535       8        1   0.003121
19    -0.021666 -0.049256       9        1   0.001628
17    -0.018123 -0.050030      10        1   0.000415
21    -0.018123 -0.050030      11        1   0.000402
6     -0.018123 -0.050030      12        1   0.000398
29    -0.018123 -0.050030      13        1   0.000395
22    -0.018123 -0.050030      14        1   0.000395
18    -0.018123 -0.050030      15        1   0.000391
4     -0.018123 -0.050030      16        1   0.000386
12    -0.018123 -0.050030      17        1   0.000386
3     -0.018123 -0.050030      18        1   0.000385
23    -0.018123 -0.050030      19        1   0.000385
14    -0.018123 -0.050030      20        1   0.000384
15    -0.018123 -0.050030      21        1   0.000383
28    -0.018123 -0.050030      22        1   0.000382
26    -0.018123 -0.050030      23        1   0.000382
24    -0.018123 -0.050030      24        1   0.000382
20    -0.018123 -0.050030      25        1   0.000382
25    -0.018123 -0.050030      26        1   0.000381
8     -0.018123 -0.050030      27        1   0.000381
1     -0.018123 -0.050030      28        1   0.000380
10    -0.018123 -0.050030      29        1   0.000374
13    -0.018123 -0.050030      30        1   0.000365, topic_info=          Term         Freq        Total Category  logprob  loglift
2154     trump  9385.000000  9385.000000  Default   30.000  30.0000
1398      news  1400.000000  1400.000000  Default   29.000  29.0000
1875      show  1643.000000  1643.000000  Default   28.000  28.0000
666   election  4430.000000  4430.000000  Default   27.000  27.0000
2226      vote  4382.000000  4382.000000  Default   26.000  26.0000
...        ...          ...          ...      ...      ...      ...
23         200     0.000082    30.935122  Topic30  -10.984  -0.3155
26        2004     0.000082     6.270618  Topic30  -10.984   1.2805
27        2005     0.000082     8.408874  Topic30  -10.984   0.9871
28        2018     0.000082    60.858105  Topic30  -10.984  -0.9921
29        2025     0.000082    10.108462  Topic30  -10.984   0.8030

[1656 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
0          1  0.055095     1
0          2  0.559645     1
0          3  0.382762     1
1          1  0.205524   100
1          2  0.393811   100
...      ...       ...   ...
2325       2  0.159275  year
2325       3  0.387016  year
8104       5  0.953116    ͜ʖ
8105       2  1.007904    ͡°
13186      8  1.150512    🇺🇸

[609 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 1, 3, 12, 8, 17, 10, 6, 20, 18, 22, 7, 30, 23, 19, 5, 13, 4, 24, 15, 16, 29, 27, 25, 21, 26, 9, 2, 11, 14])

In [18]:
leftwing_id2word, leftwing_corpus = freq_vec(leftwing_dw)

NameError: name 'leftwing_dw' is not defined

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=leftwing_corpus,
                                           id2word=leftwing_id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, leftwing_corpus, leftwing_id2word, mds="mmds", R=30)
vis